# Data Modeling 

**Ojective:** Train and store optimized regression models for each currency pair, Build a new dictionary for parsing VOL and FD.

1. Combine and clean 40 hours of the past currency datasets, and use 400 data points for model training. 
2. Loop through each currency pair, train and store models in the **models** folder.
3. Within the loop, build the dictionary for the trailing stops use. 

In [8]:
# Import required libraries
import datetime
import time
from polygon import RESTClient
from sqlalchemy import create_engine 
from sqlalchemy import text
import pandas as pd
from math import sqrt
from math import isnan
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from math import floor
import numpy as np
import pandas as pd

# import pycaret
from pycaret.regression import *

In [9]:
currency_pairs = [["EUR","USD",[], []],
                  ["GBP","USD",[], []],
                  ["USD","CHF",[], []],
                  ["USD","CAD",[], []],
                  ["USD","HKD",[], []],
                  ["USD","AUD",[], []],
                  ["USD","NZD",[], []],
                  ["USD","SGD",[], []]]

In [10]:
# define a dictionary that we are going to use for storing sorted vol and fd parsing

dic_vol = {}
dic_fd = {}

In [11]:
# create engine to connect with 4 currency pairs database. 

engine1 = create_engine("sqlite+pysqlite:///../data/day1.db", echo=False, future=True)
engine2 = create_engine("sqlite+pysqlite:///../data/trailing.db", echo=False, future=True)
engine3 = create_engine("sqlite+pysqlite:///../data/day1_unclean.db", echo=False, future=True)
engine4 = create_engine("sqlite+pysqlite:///../data/day2_unclean.db", echo=False, future=True)

## Individual Sort Method

In [12]:
# create connections with 4 database, combine them and create regression models out of 40 hours of data.
with engine1.connect() as conn1:
    with engine2.connect() as conn2:
        with engine3.connect() as conn3:
            with engine4.connect() as conn4:
                for curr in currency_pairs:
                    df1 = pd.read_sql_table(curr[0]+curr[1]+"_agg",conn1)
                    df2 = pd.read_sql_table(curr[0]+curr[1]+"_agg",conn2)
                    df3 = pd.read_sql_table(curr[0]+curr[1]+"_agg",conn3)
                    df4 = pd.read_sql_table(curr[0]+curr[1]+"_agg",conn4)
                # clean up database1 and put in a dataframe
                    df1 = df1[['avgfxrate', 'vol','fd','return_r']]
                    df1 = df1.iloc[1:]
                    df1= df1.reset_index(drop=True)
                # clean up database2 and put in a dataframe
                    df2 = df2[['avgfxrate', 'vol','fd','return_r']]
                    df2 = df2.iloc[1:]
                    df2= df2.reset_index(drop=True) 
                # clean up database3 and put in a dataframe (VOL database3 is unconverted, and need to divide them by the mean)
                    df3 = df3[['avgfxrate', 'vol','fd','return_r']]
                    df3 = df3.iloc[1:]
                    df3= df3.reset_index(drop=True)
                    # change the VOL
                    df3['vol'] = df3['vol']/df3['avgfxrate']
                # clean up database4 and put in a dataframe (VOL database3 is unconverted, and need to divide them by the mean)
                    df4 = df4[['avgfxrate', 'vol','fd','return_r']]
                    df4 = df4.iloc[1:]
                    df4= df4.reset_index(drop=True)
                    # change the VOL
                    df4['vol'] = df4['vol']/df4['avgfxrate']
           
                # concatenate 4 database into our training dataframe
                    training = pd.concat([df1, df2,df3, df4], ignore_index=True)
                   
                # sort the traning set by vol and fd
                    training_vol = training.sort_values(by=['vol'], ascending=True)
                    training_vol.reset_index(drop=True, inplace=True)

                    training_fd = training.sort_values(by=['fd'], ascending=True)
                    training_fd.reset_index(drop=True, inplace=True)

                # init two varibale threshold 1 and 2 to note down the two breaking point, and put into the dictionary.
                    thres1_vol = training_vol['vol'][132]
                    thres2_vol = training_vol['vol'][264]

                    thres1_fd = training_fd['fd'][132]
                    thres2_fd = training_fd['fd'][264]
                
                # put threshold into dictionaries
                    dic_vol[curr[0]+curr[1]] = [thres1_vol, thres1_vol]
                    dic_fd[curr[0]+curr[1]] = [thres1_fd, thres2_fd]

                # define a function to put a new series to our training datasets
                    def parse_vol(series):
                        global thres1_vol
                        global thres2_vol
                        if series <= thres1_vol:
                            return 1
                        if series <= thres2_vol:
                            return 2
                        else:
                            return 3

                    def parse_fd(series):
                        global thres1_fd
                        global thres2_fd
                        if series <= thres1_fd:
                            return 1
                        if series <= thres2_fd:
                            return 2
                        else:
                            return 3

                    training['vol_rank'] = training['vol'].apply(parse_vol)
                    training['fd_rank'] = training['fd'].apply(parse_fd)

                # After coding to vol and fd in to [1,2,3], we need multiple the reurn by 100,000 to normalize the training output
                    training['return_r_label'] = training['return_r'].apply(lambda x: x * 100000)

                
                # Assign to our newly proccessed training dataset
                    training = training[['avgfxrate', 'vol_rank', 'fd_rank','return_r_label']]

                # we will init our categorical features and numeric features into a list
                    cate_features = ['vol_rank','fd_rank']
                    num_features = ['avgfxrate']

                # Now starting to model with Pycaret regression 
                    s = setup(data=training, target='return_r_label', categorical_features=cate_features, numeric_features=num_features)
                # we have to exclude the following regression models because they don't generate predictions. 
                    best = compare_models(exclude=['llar','dummy','lasso', 'en'])
                    evaluate_model(best)
                # save the model
                    save_model(best,f'../models/{curr[0]}{curr[1]}')
                    


,Description,Value
0,Session id,3245
1,Target,return_r_label
2,Target type,Regression
3,Data shape,"(396, 4)"
4,Train data shape,"(277, 4)"
5,Test data shape,"(119, 4)"
6,Numeric features,1
7,Categorical features,2
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,24.5997,1252.7264,34.6862,-0.0462,2.2296,1.7675,0.0110
ridge,Ridge Regression,24.6074,1254.4057,34.7221,-0.0500,2.1152,2.5522,0.0140
omp,Orthogonal Matching Pursuit,24.6417,1254.9656,34.7333,-0.0503,2.0890,2.4462,0.0110
huber,Huber Regressor,24.6649,1257.8110,34.7660,-0.0529,2.0757,2.1064,0.0160
lr,Linear Regression,24.7787,1269.9979,34.9444,-0.0641,2.0537,2.4487,0.0380
lar,Least Angle Regression,24.7787,1269.9980,34.9444,-0.0641,2.0537,2.4487,0.0120
par,Passive Aggressive Regressor,26.0637,1333.8918,35.8801,-0.1320,1.4144,3.9344,0.0110
ada,AdaBoost Regressor,26.5236,1432.0228,37.2303,-0.2259,1.6200,3.4841,0.0230
lightgbm,Light Gradient Boosting Machine,27.4650,1412.4776,37.0739,-0.2413,1.3787,4.7968,0.0130
gbr,Gradient Boosting Regressor,27.5933,1481.9404,37.9652,-0.2887,1.4051,4.1595,0.0180


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Transformation Pipeline and Model Successfully Saved


,Description,Value
0,Session id,8009
1,Target,return_r_label
2,Target type,Regression
3,Data shape,"(396, 4)"
4,Train data shape,"(277, 4)"
5,Test data shape,"(119, 4)"
6,Numeric features,1
7,Categorical features,2
8,Preprocess,True
9,Imputation type,simple


...

Transformation Pipeline and Model Successfully Saved


,Description,Value
0,Session id,2421
1,Target,return_r_label
2,Target type,Regression
3,Data shape,"(396, 4)"
4,Train data shape,"(277, 4)"
5,Test data shape,"(119, 4)"
6,Numeric features,1
7,Categorical features,2
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,17.4180,553.7250,23.1176,-0.0118,1.8258,1.2620,0.0100
ridge,Ridge Regression,17.3897,554.6562,23.1339,-0.0130,1.8540,1.2720,0.0170
br,Bayesian Ridge,17.4923,558.3340,23.2391,-0.0254,2.0378,1.0843,0.0110
lr,Linear Regression,17.4681,558.7855,23.2473,-0.0266,1.8267,1.2853,0.0350
lar,Least Angle Regression,17.4681,558.7855,23.2473,-0.0266,1.8267,1.2853,0.0100
huber,Huber Regressor,17.4936,560.2345,23.2815,-0.0303,1.7895,1.4610,0.0110
par,Passive Aggressive Regressor,19.1252,654.8266,24.9716,-0.1757,1.1400,2.8570,0.0110
ada,AdaBoost Regressor,18.5035,637.7405,24.8830,-0.1936,1.5708,1.5424,0.0200
lightgbm,Light Gradient Boosting Machine,18.6821,641.7074,25.0139,-0.2042,1.4039,2.1043,0.0120
knn,K Neighbors Regressor,19.5374,717.0812,26.3617,-0.3339,1.2529,2.5906,0.0130


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Transformation Pipeline and Model Successfully Saved


In [13]:
# check and see our dictionary
dic_vol

{'EURUSD': [0.0004281211142476265, 0.0004281211142476265],
 'GBPUSD': [0.0004966384844507656, 0.0004966384844507656],
 'USDCHF': [0.00042780905428066956, 0.00042780905428066956],
 'USDCAD': [0.0003699360646692225, 0.0003699360646692225],
 'USDHKD': [6.876542096790575e-05, 6.876542096790575e-05],
 'USDAUD': [0.000666613601077178, 0.000666613601077178],
 'USDNZD': [0.0005482998171061228, 0.0005482998171061228],
 'USDSGD': [0.0002951453297140402, 0.0002951453297140402]}

In [14]:
dic_fd

{'EURUSD': [853594.7712419365, 1547680.8777432258],
 'GBPUSD': [666361.329745896, 1212260.5721637346],
 'USDCHF': [944561.0899999292, 2031250.0000000827],
 'USDCAD': [866338.8172116696, 1776093.2312128684],
 'USDHKD': [877566.5399239843, 10200033.39031332],
 'USDAUD': [592503.5178437203, 1376526.7708181816],
 'USDNZD': [294972.55499312916, 670718.232044248],
 'USDSGD': [1299890.5394999508, 2774999.999998765]}